In [3]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [33]:
ranking_egresos_nacionales_deis = pl.read_csv(
    "../data/external/ranking_nacional_egresos.csv", encoding="latin-1", separator=";"
)

In [34]:
df_grd = pl.read_csv(
    "../data/processed/df_procesada.csv",
    separator=";",
    columns=["ESTANCIA", "COD_HOSPITAL", "DIAGNOSTICO1", "ANIO_EGRESO"],
)

df_grd = df_grd.with_columns(
    pl.col("DIAGNOSTICO1").str.replace(".", "", literal=True).str.pad_end(4, "X")
)

In [35]:
ranking_egresos_nacionales_grd = (
    df_grd.group_by(pl.col(["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"]))
    .agg([pl.count().alias("n_egresos"), pl.col("ESTANCIA").sum().alias("dias_estada_totales")])
    .sort(by=["ANIO_EGRESO", "DIAGNOSTICO1", "n_egresos"], descending=True)
    .with_columns(
        pl.col("n_egresos")
        .rank(method="min", descending=True)
        .over(["ANIO_EGRESO", "DIAGNOSTICO1"])
        .alias("ranking_egresos")
    )
)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_18376\1149079150.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg([pl.count().alias("n_egresos"), pl.col("ESTANCIA").sum().alias("dias_estada_totales")])


In [31]:
comparacion_grd_deis = ranking_egresos_nacionales_grd.join(
    ranking_egresos_nacionales_deis,
    left_on=["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"],
    right_on=["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "DIAG1"],
    how="inner",
    suffix="_DEIS"
)

In [32]:
comparacion_grd_deis

ANIO_EGRESO,COD_HOSPITAL,DIAGNOSTICO1,n_egresos,dias_estada_totales,ranking_egresos,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,n_egresos_DEIS,dias_estada_totales_DEIS,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,u32,i64,u32,str,str,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
2020,116110,"""Z768""",5,4,4,"""Hospital San José (Parral)""","""Cap.21 FACTORES QUE INFLUYEN …","""Z69-Z76 PERSONAS QUE ENTRAN E…","""Z76 PERSONAS EN CONTACTO CON L…","""Persona en contacto con los se…",24,85,0,0,2,299,0.080268,1.0,150.0,0.16,null,null,null,1.0,77.0,0.311688,null,null,null
2020,105100,"""Z768""",1,1,8,"""Hospital San Juan de Dios (La …","""Cap.21 FACTORES QUE INFLUYEN …","""Z69-Z76 PERSONAS QUE ENTRAN E…","""Z76 PERSONAS EN CONTACTO CON L…","""Persona en contacto con los se…",3,16,1,0,21,299,0.010033,11.0,150.0,0.02,null,null,null,6.0,77.0,0.038961,null,null,null
2020,125100,"""Z768""",1,3,8,"""Hospital Regional (Coihaique)""","""Cap.21 FACTORES QUE INFLUYEN …","""Z69-Z76 PERSONAS QUE ENTRAN E…","""Z76 PERSONAS EN CONTACTO CON L…","""Persona en contacto con los se…",3,10,1,0,21,299,0.010033,11.0,150.0,0.02,null,null,null,6.0,77.0,0.038961,null,null,null
2020,107101,"""Z768""",1,2,8,"""Hospital San Martín (Quillota)""","""Cap.21 FACTORES QUE INFLUYEN …","""Z69-Z76 PERSONAS QUE ENTRAN E…","""Z76 PERSONAS EN CONTACTO CON L…","""Persona en contacto con los se…",1,2,0,0,34,299,0.003344,21.0,150.0,0.006667,null,null,null,10.0,77.0,0.012987,null,null,null
2020,104100,"""Z768""",10,53,3,"""Hospital San José del Carmen (…","""Cap.21 FACTORES QUE INFLUYEN …","""Z69-Z76 PERSONAS QUE ENTRAN E…","""Z76 PERSONAS EN CONTACTO CON L…","""Persona en contacto con los se…",1,1,0,0,34,299,0.003344,21.0,150.0,0.006667,null,null,null,10.0,77.0,0.012987,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2019,112102,"""A010""",1,7,6,"""Hospital de Niños Dr. Luis Cal…","""Cap.01 CIERTAS ENFERMEDADES I…","""A00-A09 ENFERMEDADES INFECCIO…","""A01 FIEBRES TIFOIDEA Y PARATIF…","""Fiebre tifoidea""",1,7,0,0,13,85,0.011765,9.0,63.0,0.015873,null,null,null,5.0,48.0,0.020833,null,null,null
2019,116108,"""A010""",1,8,6,"""Hospital Presidente Carlos Ibá…","""Cap.01 CIERTAS ENFERMEDADES I…","""A00-A09 ENFERMEDADES INFECCIO…","""A01 FIEBRES TIFOIDEA Y PARATIF…","""Fiebre tifoidea""",1,8,0,0,13,85,0.011765,9.0,63.0,0.015873,null,null,null,5.0,48.0,0.020833,null,null,null
2019,120101,"""A010""",1,6,6,"""Complejo Asistencial Dr. Vícto…","""Cap.01 CIERTAS ENFERMEDADES I…","""A00-A09 ENFERMEDADES INFECCIO…","""A01 FIEBRES TIFOIDEA Y PARATIF…","""Fiebre tifoidea""",1,6,0,0,13,85,0.011765,9.0,63.0,0.015873,null,null,null,5.0,48.0,0.020833,null,null,null
